# Select database (i.e. staging, production, local authenticated, local unauthenticated)

In [12]:
import requests
from obi_auth import get_token
from entitysdk import Client, ProjectContext, models
from enum import Enum

class DatabaseEnvironment(Enum):
    STAGING_DATABASE = "staging"
    # PRODUCTION_DATABASE = "production"
    # LOCAL_UNAUTHENTICATED = "local_unauthenticated"
    # Note: LOCAL_AUTHENTICATED is not supported by entitycore because the project/vlab ids do not exist in your keycloak groups

obi_one_api_url = "http://127.0.0.1:8100"

add_morphology_asset = False

database_environment = DatabaseEnvironment.STAGING_DATABASE
# database_environment = DatabaseEnvironment.PRODUCTION_DATABASE
# database_environment = DatabaseEnvironment.LOCAL_UNAUTHENTICATED

if database_environment == DatabaseEnvironment.STAGING_DATABASE:
    """
    - Uncomment in obi-one/.env.run-local before "make run-local": 
        export ENTITYCORE_URL=https://staging.openbraininstitute.org/api/entitycore
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"
    virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    project_id="2720f785-a3a2-4472-969d-19a53891c817"

# elif database_environment == DatabaseEnvironment.PRODUCTION_DATABASE:
#     """
#     For future: EntityCore not currently in production.
#     - Uncomment in obi-one/.env.run-local before "make run-local": 
#         export ENTITYCORE_URL=https://www.openbraininstitute.org/api/entitycore
#     """
#     entitycore_api_url = "https://www.openbraininstitute.org/api/entitycore"

elif database_environment == DatabaseEnvironment.LOCAL_UNAUTHENTICATED:
    """
    Not yet tested.
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk)
    """
    entitycore_api_url = "http://127.0.0.1:8000"
    add_morphology_asset = True
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730"
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6"
    # virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    # project_id="2720f785-a3a2-4472-969d-19a53891c817"
    
else:
    raise ValueError(f"Unsupported environment: {database_environment}")


token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

# Find a morphology

In [13]:
from entitysdk.models.morphology import (
    ReconstructionMorphology,
)
kwargs = {}
reconstruction_morphologies = client.search_entity(
    entity_type=ReconstructionMorphology, query=kwargs, limit=10
).all()

reconstruction_morphology_id = reconstruction_morphologies[4].id
print(reconstruction_morphology_id)

0fa285f5-7989-45f8-ab6d-9ff5128b15e0


# Fetch a morphology

In [14]:
from entitysdk.models.morphology import (
    ReconstructionMorphology,
)
kwargs = {}
reconstruction_morphology = client.get_entity(
    entity_id=reconstruction_morphology_id, entity_type=ReconstructionMorphology
    # entity_type=ReconstructionMorphology, query=kwargs, limit=10
)

# reconstruction_morphology_id = reconstruction_morphologies[1].id
print(reconstruction_morphology)

assets=[Asset(id=UUID('595ecfc0-69a5-4abe-abe1-d69343664f19'), update_date=None, creation_date=None, path='481_171205S1C1N3_with_contour.asc', full_path='public/c16be625-0049-49ff-bcb9-b83833057e12/3459ba2a-148f-4027-bb22-8c7202d81276/assets/reconstruction_morphology/0fa285f5-7989-45f8-ab6d-9ff5128b15e0/481_171205S1C1N3_with_contour.asc', is_directory=False, content_type='application/asc', size=1371591, sha256_digest='848c5b28d17e2ed2aadc5d0c648462747c6220540c152b6b59062b8142287333', status='created', meta={}, label=None), Asset(id=UUID('e4059bbb-6539-45eb-a71c-e02f4559eeda'), update_date=None, creation_date=None, path='481_171205S1C1N3_with_contour.swc', full_path='public/c16be625-0049-49ff-bcb9-b83833057e12/3459ba2a-148f-4027-bb22-8c7202d81276/assets/reconstruction_morphology/0fa285f5-7989-45f8-ab6d-9ff5128b15e0/481_171205S1C1N3_with_contour.swc', is_directory=False, content_type='application/swc', size=1726522, sha256_digest='ee726e934a427b7a823885f5e7f4968be9c1407a696c27088643bf064

In [15]:
reconstruction_morphology_id = reconstruction_morphology.id
print(reconstruction_morphology_id)

0fa285f5-7989-45f8-ab6d-9ff5128b15e0


In [16]:
import tempfile
from pathlib import Path
import io

if add_morphology_asset:
    with tempfile.TemporaryDirectory() as tdir:

        src = Path("single_cell_simulations/components/morphologies/C060114A5.swc")
        file2 = Path(tdir, "morphology.swc")
        # print(src.read_text())
        file2.write_text(src.read_text())

        # use an in-memory buffer to upload second asset
        buffer = io.BytesIO(b"morph bytes buffer")

        asset2 = client.upload_content(
            entity_id=reconstruction_morphology_id,
            entity_type=ReconstructionMorphology,
            file_content=buffer,
            file_name="morphology.swc",
            file_content_type="application/swc",
        )

        print(asset2)


# Call the declared morphology metrics endpoint

In [21]:
# Construct the full endpoint URL
url = f"{obi_one_api_url}/declared/neuron-morphology-metrics/{reconstruction_morphology_id}"

# Set headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Make the GET request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")


Success: {'aspect_ratio': 0.4648461807482674, 'circularity': 0.6599072772975014, 'length_fraction_above_soma': 0.5042385458946228, 'max_radial_distance': 682.55810546875, 'number_of_neurites': 12, 'soma_radius': 6.802358936219697, 'soma_surface_area': 581.4722290039062}


# Call the generated morphology metrics endpoint
# ERROR WITH THE MORPHOLOGY (AS USING SWC UNLIKE THE DECLARED). TO LOOK INTO!

In [20]:
import requests

# Construct the full endpoint URL
url = f"{obi_one_api_url}/generated/morphology-metrics-run-grid"

# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Construct request body — adjust this with the actual morphology metrics form data!
# Placeholder example:
request_body = {
  "type": "MorphologyMetricsForm",
  "initialize": {
    "type": "MorphologyMetricsForm.Initialize",
    "morphology": {
      "id_str": str(reconstruction_morphology_id),
    }
  }
}

# Make the POST request
response = requests.post(url, headers=headers, json=request_body)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

Error 500: Traceback (most recent call last):
  File "/Users/james/Documents/obi/code/obi-one/.venv/lib/python3.12/site-packages/starlette/middleware/errors.py", line 165, in __call__
    await self.app(scope, receive, _send)
  File "/Users/james/Documents/obi/code/obi-one/.venv/lib/python3.12/site-packages/starlette/middleware/cors.py", line 85, in __call__
    await self.app(scope, receive, send)
  File "/Users/james/Documents/obi/code/obi-one/.venv/lib/python3.12/site-packages/starlette/middleware/exceptions.py", line 62, in __call__
    await wrap_app_handling_exceptions(self.app, conn)(scope, receive, send)
  File "/Users/james/Documents/obi/code/obi-one/.venv/lib/python3.12/site-packages/starlette/_exception_handler.py", line 53, in wrapped_app
    raise exc
  File "/Users/james/Documents/obi/code/obi-one/.venv/lib/python3.12/site-packages/starlette/_exception_handler.py", line 42, in wrapped_app
    await app(scope, receive, sender)
  File "/Users/james/Documents/obi/code/obi-on